In [10]:
from markov_chains import *

In [36]:
%run markov_chains

In [39]:
recruitment_means_list, recruitment_std_devs_list, recruitment_future_dfs_dict = calculate_markovs("./recruitment.csv", 10)

In [47]:
uncertainty_means_list, uncertainty_std_devs_list, uncertainty_future_dfs_dict = calculate_markovs("./uncertainty.csv", 1)

In [13]:
employment_growth_means_list, employment_growth_std_devs_list, employment_growth_future_dfs_dict = calculate_markovs("./employment_growth.csv", 10)

In [48]:
uncertainty_final_dfs_list: List[pd.DataFrame] = [dfs_list[-1] for dfs_list in uncertainty_future_dfs_dict.values()]
uncertainty_final_dfs_list

[         up      down
 0  0.318182  0.681818,
          up      down
 0  0.409091  0.590909,
     up  down
 0  0.5   0.5,
          up      down
 0  0.666667  0.333333,
     up  down
 0  0.4   0.6]